In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os
import glob as gb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from zipfile import ZipFile
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

In [ ]:
# Load the dataset
df_path = '/content/dataset.csv'
df = pd.read_csv(df_path)


In [ ]:
#Display the first few rows of the modified sheet
print(f'First few rows of the modified sheet:')
print(df.head())
print("\n")

In [ ]:
features = ['t/s', 'p', 'q', 'r', 'mx', 'my', 'mz']
X = df[features].values

# Extract fault values (m1, m2, m3, m4) as targets
targets = df[['m1', 'm2', 'm3', 'm4']].values

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

In [ ]:
# Normalize the data using Min-Max scaling (The minimum value of each feature becomes 0 and the maximum value becomes 1.)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Print the scaled values for inspection
print("X_train_scaled:\n", X_train_scaled)
print("\nX_test_scaled:\n", X_test_scaled)

In [ ]:
 #Reshape the input data (converts each row into a unique id-variable) (In the new shape, the first dimension remains the number of samples,


X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, RepeatVector, Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Huber

# Define the input shape and latent dimension
input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])
latent_dim = 3

# Define the input layer
input_layer = Input(shape=input_shape)

# Encoder
encoder = LSTM(100, activation='relu', return_sequences=True)(input_layer)
encoder = LSTM(75, activation='relu', return_sequences=True)(encoder)
encoder = LSTM(50, activation='relu', return_sequences=True)(encoder)
encoder = LSTM(25, activation='relu', return_sequences=True)(encoder)
encoder = LSTM(latent_dim, activation='relu')(encoder)

# Decoder
decoder = RepeatVector(X_train_reshaped.shape[1])(encoder)
decoder = LSTM(25, activation='relu', return_sequences=True)(decoder)
decoder = LSTM(50, activation='relu', return_sequences=True)(decoder)
decoder = LSTM(75, activation='relu', return_sequences=True)(decoder)
decoder = LSTM(100, activation='relu', return_sequences=True)(decoder)
decoder = LSTM(X_train_reshaped.shape[2], activation='sigmoid', return_sequences=True)(decoder)

# Define the autoencoder model
autoencoder = Model(input_layer, decoder)

# Define custom loss function - Huber Loss
def custom_huber_loss(y_true, y_pred):
    return tf.reduce_mean(Huber()(y_true, y_pred))

# Compile the model with the custom loss
autoencoder.compile(optimizer='adam', loss=custom_huber_loss)

# Print the summary
autoencoder.summary()


In [ ]:
# Train the autoencoder and monitor metrics
history = autoencoder.fit(X_train_reshaped, [X_train_reshaped, X_train_reshaped], epochs=100, batch_size=32, validation_split=0.2, shuffle=True)

In [ ]:
# Evaluate the model on test data
loss_value = autoencoder.evaluate(X_test_reshaped, [X_test_reshaped, X_test_reshaped])

# Print the total loss value
print("Total Loss:", loss_value)


In [ ]:
import matplotlib.pyplot as plt

# Predict the reconstructed outputs
reconstructed_outputs = autoencoder.predict(X_test_reshaped)

# Choose a sample index
sample_index = 0

# Original input
original_input = X_test_reshaped[sample_index]

# Reconstructed output
reconstructed_output = reconstructed_outputs[sample_index]

# Set font
plt.rcParams['font.family'] = 'Times New Roman'

# Create a figure with higher DPI
plt.figure(figsize=(12, 6), dpi=400)

# Plot original input
plt.subplot(1, 2, 1)
plt.plot(original_input.flatten(), label='Original Input', color='blue')
plt.title('Original Input', fontsize=16)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Magnitude', fontsize=14)

# Plot reconstructed output
plt.subplot(1, 2, 2)
plt.plot(reconstructed_output.flatten(), label='Reconstructed Output', color='red')
plt.title('Reconstructed Output', fontsize=16)
plt.xlabel('Time', fontsize=14)
plt.ylabel('Magnitude', fontsize=14)

# Adjust layout
plt.tight_layout()

# Save the plot
plt.savefig('reconstruction_plot.png', dpi=400)

# Show the plot
plt.show()


In [ ]:
# Print the values of original input and reconstructed output
print("Original Input Values:", original_input.flatten())
print("Reconstructed Output Values:", reconstructed_output.flatten())


In [ ]:
from tensorflow.keras.models import Model

# Define encoder model
encoder_model = Model(inputs=input_layer, outputs=encoder)

# Encode the input data to get the latent space representations
latent_space_representation = encoder_model.predict(X_train_reshaped)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Define journal colors
tsne_color = '#D55E00'  # Orange color for t-SNE plot

# Perform t-SNE to reduce the dimensionality of the latent space representations
tsne = TSNE(n_components=2)
latent_space_tsne = tsne.fit_transform(latent_space_representation)

# Set font
plt.rcParams['font.family'] = 'Times New Roman'

# Plot the latent space representations
plt.figure(figsize=(8, 6))
plt.scatter(latent_space_tsne[:, 0], latent_space_tsne[:, 1], c=tsne_color, alpha=0.5)
plt.title('Latent Space Visualization (t-SNE)', fontsize=18)
plt.xlabel('t-SNE Dimension 1', fontsize=14)
plt.ylabel('t-SNE Dimension 2', fontsize=14)
plt.grid(False)  # Remove grid lines

# Increase tick font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Save the plot with DPI 400
plt.savefig('latent_space_tsne.png', dpi=400, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()


In [ ]:
# Print the latent space representations
print("Latent Space Representations:")
print(latent_space_representation)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Define alternative colors
training_color = '#DD8452'     # Lighter orange color for training loss
validation_color = '#8172B0'   # Light purple color for validation loss

# Collect the loss values from the History object
loss = history.history['loss']
val_loss = history.history['val_loss']

# Set font
plt.rcParams['font.family'] = 'Times New Roman'

# Create figure and axis objects
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the training and validation loss values
hist, bins = np.histogram(loss, bins=20)
x = (bins[:-1] + bins[1:]) / 2
ax.bar(x, hist, zs=0, zdir='y', width=0.7, color=training_color, alpha=0.8, label='Training Loss')

hist, bins = np.histogram(val_loss, bins=20)
x = (bins[:-1] + bins[1:]) / 2
ax.bar(x, hist, zs=1, zdir='y', width=0.7, color=validation_color, alpha=0.8, label='Validation Loss')

# Set plot title and labels
ax.set_title('Histogram of Loss Values with Huber Loss Function', fontsize=18, pad=20)
ax.set_xlabel('Loss', fontsize=14, labelpad=15)
ax.set_ylabel('Frequency', fontsize=14, labelpad=15)
ax.set_zlabel('Count', fontsize=14, labelpad=15)

# Add legend
ax.legend(fontsize=14)

# Increase tick font size
ax.tick_params(axis='both', which='major', labelsize=12, pad=8)

# Remove grid lines
ax.grid(False)

# Set transparent background
ax.patch.set_alpha(0)

# Save the plot with DPI 400
plt.savefig('loss_histogram_3d.png', dpi=400, bbox_inches='tight', transparent=True)

# Show the plot
plt.show()


In [ ]:
# Print the first few loss values
print("Training Loss Values:", loss[:5])
print("Validation Loss Values:", val_loss[:5])

# Print the mean and standard deviation of the loss values
print("Mean Training Loss:", np.mean(loss))
print("Mean Validation Loss:", np.mean(val_loss))
print("Standard Deviation of Training Loss:", np.std(loss))
print("Standard Deviation of Validation Loss:", np.std(val_loss))
